In [1]:
import requests
import pandas as pd
import re
from collections import OrderedDict
from googletrans import Translator
from mechanize import Browser
from nltk.chunk.regexp import RegexpParser
from nltk.tree import Tree
from nltk import pos_tag, word_tokenize
from gensim.models import KeyedVectors
from bs4 import BeautifulSoup

model = KeyedVectors.load_word2vec_format("~/GoogleNews-vectors-negative300.bin", binary=True)
entityID = "Q668"


In [2]:
def makeSparqlRequest(query):
    url = 'https://query.wikidata.org/sparql'
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    return data

spoQuery = """

SELECT ?p ?pHi ?pred ?object WHERE {
  wd:""" + entityID + """ ?pred ?o .

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "hi" .
    ?o rdfs:label ?object .
  }
  ?prop wikibase:directClaim ?pred .
   SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?prop rdfs:label ?p .
  }
  
  ?propHi wikibase:directClaim ?pred .
   SERVICE wikibase:label {
    bd:serviceParam wikibase:language "hi" .
    ?propHi rdfs:label ?pHi .
  }
  FILTER(LANG(?object) = "hi")

}
"""

results = makeSparqlRequest(spoQuery)

In [3]:
entityIDquery = """
SELECT * WHERE {
  wd:""" + entityID + """ rdfs:label ?label.
  FILTER(LANGMATCHES(LANG(?label), "hi"))
}
LIMIT 1
"""

results2 = makeSparqlRequest(entityIDquery)
entityName = results2["results"]["bindings"][0]["label"]["value"]
entityName

'भारत'

In [4]:



triplets = []

for item in results['results']['bindings']:
    obj = item['object']['value']
    translator = Translator()
    if item["pHi"]["value"][0] == "P":
        pred = item['p']["value"]
        predHi = translator.translate(pred, dest="hi").text
    else:
        predHi = item['pHi']["value"]
    triplets.append(OrderedDict({
        'subject': entityName,
        'predicate': predHi,
        "predicateEn": item['p']["value"],
        "propertyID": item['pred']["value"],
        'object': obj}))

df = pd.DataFrame(triplets)


In [27]:
def posTagHindi(text):
    br = Browser()
    br.open("http://taghindi.herokuapp.com/")
    br.form = br.forms()[0]
    br["text"] = text
    response = br.submit().read()

    soup = BeautifulSoup(response.decode("utf-8"))

    tags = [el.text for el in soup.find_all('span', {'style': 'color:blue'})]
    taggedText = list(zip(text.split(), tags))
    
    return taggedText

predicates = list(set([triplet["predicate"] for triplet in triplets]))
wikiDict = {pred: [] for pred in predicates}



In [29]:
def getSim(a,b):
    try:
        return model.similarity(a,b)
    except:
        return 0
    
def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        INSTANCE:   {(<JJ+>)?<PRE>}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

topics = ["history", "geography", "politics"]
for triplet in triplets:
    instances = getInstances(triplet["predicateEn"])
    sectionIdx = 0
    if instances:
        instance = "_".join(instances[-1].split(" "))
        s1 = getSim(instance, topics[0])
        s2 = getSim(instance, topics[1])
        s3 = getSim(instance, topics[2])
        cutoff = [el for el in [s1,s2,s3] if el>0.14]
        if cutoff:
            sectionIdx = [s1,s2,s3].index(cutoff[0]) + 1
    wikiDict[triplet["predicate"]].append((triplet["object"], sectionIdx))


In [21]:
wikiDict['उदहारण है'][-1][0]

'सम्प्रभु राज्य'

In [ ]:
translator = Translator()
translator.translate("hello", dest="hi")

In [ ]:
for elem in wikiDict:
    taggedElem = posTagHindi(elem)
    print (taggedElem)

In [33]:
from bs4 import BeautifulSoup
article = ""
articleSections = ["", "इतिहास\n\n", "भूगोल\n\n", "राजनीति\n\n"]
for verb_str in wikiDict:
    taggedElem = posTagHindi(verb_str)
    sentence = ""
    subject_str = entityName
    sectionNum = wikiDict[verb_str][0][1]
    if len(wikiDict[verb_str])>1:
        object_str = ", ".join([el[0] for el in wikiDict[verb_str][:-1]]) + " और " + wikiDict[verb_str][-1][0]
    else:
        object_str = wikiDict[verb_str][0][0]
    
    if taggedElem[-1] == "VM":
        sentence = " ".join([subject_str, object_str, "का", verb_str])
    elif taggedElem[-1] == "VAUX":
        sentence = " ".join([subject_str, object_str, "के साथ", verb_str])
    elif "NN" in taggedElem[-1] or taggedElem[-1] == "XC":
        sentence = " ".join([subject_str, "का", verb_str,  object_str])
    else:
        sentence = " ".join([subject_str, object_str, verb_str])
        

    if sentence.split(" ")[-1] != "है":
        sentence += " है"

    sentence += "| "
    articleSections[sectionNum] += sentence

del articleSections[2]


TypeError: sequence item 0: expected str instance, list found

In [36]:
article = "\n\n\n\n".join(articleSections)
open("article.txt","w+").write(article)

4119

In [35]:
print (article)

भारत का उच्चतम डोमेन .in, .ਭਾਰਤ, .ભારત, .भारत, .ଭାରତ और .ভাৰত है| भारत भारतीय रिज़र्व बैंक केंद्रीय अधिकोष है| भारत का कुल चिह्न भारत का राजचिन्ह है| भारत श्रेणी:भारत विषय की मुख्य श्रेणी है| भारत का प्रशासनिक इकाई गुजरात, आन्ध्र प्रदेश, अरुणाचल प्रदेश, असम, बिहार, छत्तीसगढ़, गोआ, हरियाणा, हिमाचल प्रदेश, जम्मू और कश्मीर, झारखण्ड, कर्नाटक, केरल, मध्य प्रदेश, महाराष्ट्र, मणिपुर, मेघालय, दिल्ली, पश्चिम बंगाल, त्रिपुरा, राजस्थान, तमिल नाडु, उत्तर प्रदेश, उत्तराखण्ड, मिज़ोरम, सिक्किम, नागालैण्ड, ओडिशा, पंजाब, लक्षद्वीप, अण्डमान और निकोबार द्वीपसमूह, चण्डीगढ़, दादर और नागर हवेली, दमन और दीव, पुदुच्चेरी और तेलंगाना है| भारत का महत्वपूर्ण घटना भारतीय स्वतन्त्रता आन्दोलन है| भारत का कार्यकारी मंडल भारतीय संसद है| भारत सिन्धु नदी द्वारा नामांकित है| भारत का उच्चतम न्यायिक प्राधिकरण भारत का उच्चतम न्यायालय है| भारत का विकिमीडिया रूपरेखा भारत सारावली है| भारत यू टी सी + ५:३० और भारतीय मानक समय समय क्षेत्र में स्थित है| भारत का राजधानी नई दिल्ली है| भारत का ध्वज भारत का ध्वज है| भारत ब्रिटिश राज पि

In [30]:
wikiDict

{'उच्चतम डोमेन': [('.in', 0),
  ('.ਭਾਰਤ', 0),
  ('.ભારત', 0),
  ('.भारत', 0),
  ('.ଭାରତ', 0),
  ('.ভাৰত', 0)],
 'उदहारण है': [('देश', 1),
  ('गणराज्य', 1),
  ('स्वायत्त्योपनिवेश', 1),
  ('सम्प्रभु राज्य', 1)],
 'शासनाध्यक्ष': [('नरेन्द्र मोदी', 3)],
 'कार्यालय सरकार के मुखिया द्वारा आयोजित': [('भारत का प्रधानमन्त्री', 3)],
 'राज्याध्यक्ष': [('राम नाथ कोविन्द', 1)],
 'केंद्रीय अधिकोष': [('भारतीय रिज़र्व बैंक', 0)],
 'का भाग': [('दक्षिण पूर्वी एशियाई राष्ट्रों का संगठन', 1),
  ('राष्ट्रकुल', 1),
  ('गुट निरपेक्ष आंदोलन', 1),
  ('दक्षिण एशियाई क्षेत्रीय सहयोग संगठन', 1),
  ('ब्रिक', 1),
  ('दक्षिण एशिया', 1)],
 'राष्ट्रगान': [('जन गण मन', 1)],
 'सरकार का प्रकार': [('संघीय गणराज्य', 3)],
 'कुल चिह्न': [('भारत का राजचिन्ह', 0)],
 'विषय की मुख्य श्रेणी': [('श्रेणी:भारत', 0)],
 'प्रशासनिक इकाई': [('गुजरात', 0),
  ('आन्ध्र प्रदेश', 0),
  ('अरुणाचल प्रदेश', 0),
  ('असम', 0),
  ('बिहार', 0),
  ('छत्तीसगढ़', 0),
  ('गोआ', 0),
  ('हरियाणा', 0),
  ('हिमाचल प्रदेश', 0),
  ('जम्मू और कश्मीर', 0),
  (